# Tarea 7

_175904 - Jorge III Altamirano Astorga_

## Librerías y Carga de Datos

In [197]:
#importo librerías
import pyspark
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.functions import *
from pyspark.sql import DataFrameStatFunctions, DataFrame
from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import *
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import re as re
import time

In [ ]:
#arranque de Spark
conf = SparkConf()
conf.set("spark.driver.memory", "16g")
conf.set("spark.driver.cores", 4)
conf.set("spark.driver.memoryOverhead", 0.9)
conf.set("spark.executor.memory", "32g")
conf.set("spark.executor.cores", 12)
conf.set("spark.jars", "/home/jaa6766")
sc = SparkContext(master = "local[14]", sparkHome="/usr/local/spark/", 
                  appName="tarea-mge-7", conf=conf)
spark = SQLContext(sc)

In [3]:
#carga de datos
flights = spark.read.csv("hdfs://172.17.0.2:9000/data/flights/flights.csv", 
                         inferSchema=True, 
                         header=True)
flights = flights.cache()
flights.show(2)
flights.printSchema()

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

In [53]:
#columnas más relevantes
relevant_cols = ["YEAR", "MONTH", "DAY", "DAY_OF_WEEK", "AIRLINE", 
                "FLIGHT_NUMBER", "ORIGIN_AIRPORT", "DESTINATION_AIRPORT",
                "SCHEDULED_DEPARTURE", "DEPARTURE_TIME", "SCHEDULED_TIME",
                "AIR_TIME", "DISTANCE", "SCHEDULED_ARRIVAL", "CANCELLED",
                "DEPARTURE_DELAY"]
relevant_cols

['YEAR',
 'MONTH',
 'DAY',
 'DAY_OF_WEEK',
 'AIRLINE',
 'FLIGHT_NUMBER',
 'ORIGIN_AIRPORT',
 'DESTINATION_AIRPORT',
 'SCHEDULED_DEPARTURE',
 'DEPARTURE_TIME',
 'SCHEDULED_TIME',
 'AIR_TIME',
 'DISTANCE',
 'SCHEDULED_ARRIVAL',
 'CANCELLED',
 'DEPARTURE_DELAY']

In [5]:
### selección de columnas
flights = flights.select(relevant_cols)
### descartar nulos
flights = flights.na.drop().cache()
### dividir en sets de entrenamiento y set de pruebas
flights.write.parquet("hdfs://172.17.0.2:9000/tmp/flights-tmp.parquet", mode="overwrite")
flights.show(2)

+----+-----+---+-----------+-------+-------------+--------------+-------------------+-------------------+--------------+--------------+--------+--------+-----------------+---------+---------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|SCHEDULED_TIME|AIR_TIME|DISTANCE|SCHEDULED_ARRIVAL|CANCELLED|DEPARTURE_DELAY|
+----+-----+---+-----------+-------+-------------+--------------+-------------------+-------------------+--------------+--------------+--------+--------+-----------------+---------+---------------+
|2015|    1|  1|          4|     AS|           98|           ANC|                SEA|                  5|          2354|           205|     169|    1448|              430|        0|            -11|
|2015|    1|  1|          4|     AA|         2336|           LAX|                PBI|                 10|             2|           280|     263|    2330|              750|        0|             -8|
+----+----

## Crear sets de Entrenamiento y Pruebas

Aquí también se tienen los objetos que se utilizarán en el pipeline más adelante.

In [54]:
%%time
#por eficiencia he visto que es mejor guardarlo
flights = spark.read.parquet("hdfs://172.17.0.2:9000/tmp/flights-tmp.parquet")
flights = flights.cache()
#dividir el set
(train, test) = flights.randomSplit([0.7, 0.3], seed=175904)
train2 = train.sample(fraction=0.001, seed=175904)

#guardamos por eficiencia, nuevamente
train.write.parquet("hdfs://172.17.0.2:9000/tmp/train-tmp.parquet", mode="overwrite")
test.write.parquet("hdfs://172.17.0.2:9000/tmp/test-tmp.parquet", mode="overwrite")
train = spark.read.parquet("hdfs://172.17.0.2:9000/tmp/train-tmp.parquet")
test = spark.read.parquet("hdfs://172.17.0.2:9000/tmp/test-tmp.parquet")
train = train.cache()
test = test.cache()

#descartando categóricas y la que vamos a predecir: "DEPARTURE_DELAY"
features = [ x for x in train.schema.names if x not in 
            [ "DEPARTURE_DELAY", "AIRLINE", "ORIGIN_AIRPORT", "DESTINATION_AIRPORT"] ]
#agregando las columnas que ya van a ser indizadas
features.append("airline_indexer")
features.append("origin_indexer")
features.append("destination_indexer")
print("Features que se tomarán para la predicción: ", features)

#indización de columnas: de categóricas a texto
li0 = StringIndexer(inputCol="AIRLINE",
                    outputCol="airline_indexer", 
                    handleInvalid="skip") \
    .fit(flights)
li1 = StringIndexer(inputCol="ORIGIN_AIRPORT",
                    outputCol="origin_indexer", 
                    handleInvalid="skip") \
    .fit(flights)
li2 = StringIndexer(inputCol="DESTINATION_AIRPORT",
                    outputCol="destination_indexer", 
                    handleInvalid="skip") \
    .fit(flights)
li  = StringIndexer(inputCol="DEPARTURE_DELAY",
                    outputCol="delay_indexer", 
                    handleInvalid="skip") \
    .fit(flights)
#este ensamble es requerido para usar los clasificadores y regresores de Spark ML
va0 = VectorAssembler() \
    .setInputCols(features) \
    .setOutputCol("features")
pca = PCA(k=10, inputCol="features", outputCol="features_pca")
lc = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                   labels=li.labels)


Features que se tomarán para la predicción:  ['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'FLIGHT_NUMBER', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'SCHEDULED_TIME', 'AIR_TIME', 'DISTANCE', 'SCHEDULED_ARRIVAL', 'CANCELLED', 'airline_indexer', 'origin_indexer', 'destination_indexer']
CPU times: user 59.8 ms, sys: 17 ms, total: 76.7 ms
Wall time: 6.53 s


## Pruebas y componentes ML de los pipelines

Esto no va a ir en el documento final, dado que sólo estuve _jugando_. Esto resultó necesario para entender mejor cómo funciona y calcular tiempos.

Es destacable que estos son objetos que se utilizarán en el pipeline.

In [55]:
%%time
# regresión logística
lr = LogisticRegression(featuresCol="features_pca",
                        labelCol="delay_indexer",
                        #maxIter=10, elasticNetParam=0.8,
                        regParam=0.3, 
                        family="multinomial",
                        predictionCol="prediction")
#creación del pipeline, este no se va a utilizar en el Magic Loop
pipeline1 = Pipeline(stages=[li0, li1, li2, va0, pca, li, lr, lc]) 
# parámetros que se van a utilizar en regresión logística
pgLR = ParamGridBuilder() \
    .addGrid(lr.elasticNetParam, [0.2, 0.5, 0.8]) \
    .addGrid(lr.maxIter, [2,5,10]) \
    .build()

#el número de folds es inválido, incluso no recomendado directamente en la documentación de 
#Spark, sin embargo, es un punto de inicio. 
#
#Este componente no se va a utilizar en el Magic Loop
cv1 = CrossValidator(estimator=pipeline1,
                      estimatorParamMaps=pgLR,
                      evaluator=MulticlassClassificationEvaluator(labelCol="DEPARTURE_DELAY"),
                      numFolds=2)

CPU times: user 9 ms, sys: 1.3 ms, total: 10.3 ms
Wall time: 24.6 ms


In [56]:
%%time
# regresión lineal
glr = GeneralizedLinearRegression(featuresCol="features_pca",
                        labelCol="delay_indexer", family="Gaussian")
# parámetros que se van a utilizar para regresión lineal
pgGLR = ParamGridBuilder() \
    .addGrid(glr.family, ["Gaussian", "Poisson", "Tweedie"]) \
    .addGrid(glr.maxIter, [10, 25, 50]) \
    .build()
#creación del pipeline, este no se va a utilizar en el Magic Loop
pipeline2 = Pipeline(stages=[li0, li1, li2, va0, pca, li, glr, lc])
#el número de folds es inválido, incluso no recomendado directamente en la documentación de 
#Spark, sin embargo, es un punto de inicio. 
#
#Este componente no se va a utilizar en el Magic Loop
cv2 = CrossValidator(estimator=pipeline2,
                          estimatorParamMaps=pgGLR,
                          evaluator=MulticlassClassificationEvaluator(labelCol="DEPARTURE_DELAY"),
                          numFolds=2)

CPU times: user 10.4 ms, sys: 0 ns, total: 10.4 ms
Wall time: 27.3 ms


## Magic Loop

Aquí definí la función de Magic Loop. Le llamé magic_loop3 dado que es la función que nombramos y utilizamos el semestre pasado Augusto Sagón era magic_loop2.

### Aplicación del Magic Loop

In [195]:
%%time
DEBUG = False
def magic_loop3(pipelines, grid, train, test, cvfolds=3):
    best_score = 0.0 #symbolic high value :-)
    best_grid = None #inicializar la variable
    #este loop inicia las pruebas secuenciales de los pipelines:
    #es relevante que no sólo soporta 2, sino se va en cada uno 
    #de los que estén presentes en la lista
    for pipe in pipelines:
        try:
            #quiero que se desligue, para no modificar (al final, usa poco RAM)
            pipe = pipe.copy()
            #etapas del pipeline
            stages = pipe.getStages()
            #obtener el predictor (el motor ML)
            predictor = [stage for stage in stages
                if "pyspark.ml.classification" in str(type(stage)) or
                 "pyspark.ml.regression" in str(type(stage))][0]
            predictor_i = stages.index(predictor)
            stringer = [stage for stage in pipe.getStages()
                if "pyspark.ml.feature.StringIndexer" in str(type(stage))][0]
            if DEBUG: print("pipeline:\n%s\n\n"%stages)
            if DEBUG: print("predictor=%s (index %s, type %s), stringer=%s (%s)\n"%
                  (predictor, stages.index(predictor), type(predictor),
                  stringer, type(stringer)))
            #dado que no son predicciones susceptibles a cambios en el CV
            prepipe = Pipeline(stages=stages[0:(predictor_i)])
            if DEBUG: print("pre pipeline:\n%s\n\n"%prepipe.getStages())
            print("Starting fit on prepipe...")
            #modelo para el prepipeline
            prepipem = prepipe.fit(train)
            print("Starting transform on prepipe...")
            train2 = prepipem.transform(train)
            test2 = prepipem.transform(test)
            #el motor ML sí es susceptible CV
            postpipe = Pipeline(stages=stages[(predictor_i):len(stages)])
            if DEBUG: print("post pipeline:\n%s\n\n"%postpipe.getStages())
            #creación del Cross Validator
            gridcv = CrossValidator(estimator=postpipe,
                          estimatorParamMaps=grid,
                          evaluator=MulticlassClassificationEvaluator(labelCol="DEPARTURE_DELAY"),
                          numFolds=cvfolds)
            #extraemos el nombre y le quitamos la parte "fea" que devuelve type()
            predictr = [str(type(stage)) for stage in pipe.getStages() 
            if "pyspark.ml.classification" in str(type(stage)) or
             "pyspark.ml.regression" in str(type(stage))][0]
            predictr = re.sub("^<class 'pyspark\.ml\.(classification|regression)\.([a-zA-Z0-9]+)'>", 
                              "\\2", 
                              predictr)
            #creación del modelo
            print("Starting fit on %s..."%predictr)
            gridcvm = gridcv.fit(train2)
            #aplicación del modelo
            print("Starting transform on %s..."%predictr)
            preds = gridcvm.transform(test2)
            #obtenemos el evaluador para el uso en la medición de errores
            ev = gridcvm.getEvaluator()
            #obtenemos la métrica del error
            metric = ev.getMetricName()
            print("Starting error calculation on %s..."%predictr)
            #obtenemos el valor del eror
            error  = ev.evaluate(preds)
            print("Error %s: %f"% (metric, error))
            #si es mejor que el modelo pasado, lo guardamos. El último
            #guardado será el que devuelva esta función
            if error > best_score:
                print("%s is the best model so far: %f (%s)"%(predictr, error, metric))
                best_grid = gridcvm
        #manejo de errores y horrores
        except Exception as e:
            print('Error during Magic Loop:', e)
        continue
    return best_grid

paramGrid = ParamGridBuilder() \
                .addGrid(glr.family, ["Gaussian", "Poisson", "Tweedie"]) \
                .addGrid(glr.maxIter, [1, 2, 3]) \
                .addGrid(lr.maxIter, [1, 2, 3]) \
                .addGrid(lr.elasticNetParam, [0.1,0.2,0.3]) \
                .build()
magic = magic_loop3(pipelines, paramGrid, train, test, 3) 

Starting fit on prepipe...
Starting transform on prepipe...
Starting fit on LogisticRegression...
Starting transform on LogisticRegression...
Starting error calculation on LogisticRegression...
Error f1: 0.006251
LogisticRegression is the best model so far: 0.006251 (f1)
Starting fit on prepipe...
Starting transform on prepipe...
Starting fit on GeneralizedLinearRegression...
Starting transform on GeneralizedLinearRegression...
Starting error calculation on GeneralizedLinearRegression...
Error f1: 0.000000
CPU times: user 12min 16s, sys: 4min 22s, total: 16min 38s
Wall time: 13h 53min 23s


### Mejor Modelo

Pruebas con el mejor Modelo

In [217]:
%%time
#obtengo el pipeline que devolvió el magic loop
best_model = magic.getEstimator()
#obtenemos el paso del clasificador
best_estimator = best_model.getStages()[0]
#guardo en una variable los parámetros más adecuados
best_estimator_params = best_estimator.extractParamMap()
#obtener el evaluador para medir errores
ev0 = magic.getEvaluator()
#impresión de los parámetros y el mejor modelo
print("%s (best model) Parameters: maxIter=%d, elasticNetParam=%f"%
      (re.sub("^<class 'pyspark\.ml\.(classification|regression)\.([a-zA-Z0-9]+)'>", 
                              "\\2", (str)((type(best_estimator)))), 
       best_estimator_params[best_estimator.maxIter], 
       best_estimator_params[best_estimator.elasticNetParam]))
#aquí probé como se veían los errores
preds = magic.transform(Pipeline(stages=pipeline1.getStages()[0:6]).fit(test).transform(test))
print("Error %s: %f"% (ev0.getMetricName(), ev0.evaluate(preds)))

LogisticRegression (best model) Parameters: maxIter=100, elasticNetParam=0.000000
Error f1: 0.006251
CPU times: user 393 ms, sys: 131 ms, total: 524 ms
Wall time: 9.87 s


## Otras pruebas

El primer modelo que probé fue el de Bosques aleatorios. Esto fue sin éxito: además de que consumía mucho tiempo tenía un error muy similar a Regresión Logística. Por eso tiene un número de pipeline 0: inicié como se debe, con 0 la numeración de mis objetos. :-)

## Conclusiones


Sin duda el `Magic Loop` es una gran herramienta para iniciar _auto ML_. Sin embargoo, observé que en Machine Learning muchos modelos toman demasiado tiempo, y son prometedores y populares a utilizarse frecuentemente, como RandomForest. Los resultados son variados, por lo que sin importar el número de folds se puede tener una buena idea de cómo se comportan, siendo esto un poco de prueba y error; cosa que no permite auto ML de manera rápida, a menos que se tengan recursos ilimitados (sobretodo de tiempo) y disponibles.

Es un poco quisquilloso utilizar Spark al compararlo con el noble R, tengo que aceptarlo. Aún con mi background de Ingeniero en Sistemas. Es claro que yo inicialmente ví con malos ojos R, y quería puro Python. Me es claro que es mucho más escalable y estable (para producción). Pero R es sumamente noble para el aprendizaje, pruebas y demás; comparado con el rigor de Python (y más aún en Java/Scala). Lástima que R no es tan escalable.

Además Python no tiene ggplot que tanto nos gusta. Pero combinar las herramientas y utilizarlas sabiamente (esto después de cierto sufrimiento por batallar terminé aprendiendo, apreciando y teniendo no una visión superficial de las cosas, para poder ejercer en el campo.

Sin duda seguiré utilizando python, scikit-learn y Spark, pero también es altamente probable que siga utilizando R para hacer pruebas y pequeños modelos, dado que es sumamente rápido realizarlas. Cada herramienta a cada aplicación, aprovechándo sus fortalezas y considerando sus debilidades.

## Bibliografía

* <https://blog.insightdatascience.com/spark-pipelines-elegant-yet-powerful-7be93afcdd42>
* <https://spark.apache.org/docs/2.3.0/ml-classification-regression.html#generalized-linear-regression>
* <https://spark.apache.org/docs/latest/api/python/index.html>
* <https://spark.apache.org/docs/2.2.0/ml-pipeline.html>
* <https://stackoverflow.com/questions/37278999/logistic-regression-with-spark-ml-data-frames>
* <https://elbauldelprogramador.com/en/how-to-convert-column-to-vectorudt-densevector-spark/>
* <https://www.dezyre.com/apache-spark-tutorial/pyspark-tutorial>
* <https://github.com/apache/spark/blob/master/examples/src/main/python/ml/generalized_linear_regression_example.py>
* <https://github.com/apache/spark/blob/master/data/mllib/sample_linear_regression_data.txt>
* <https://en.m.wikipedia.org/wiki/F1_score>

In [ ]:
#sc.stop()